In [1]:
using LinearAlgebra

function cross_prod_matrix( k )
    return [ 0  -k[3] k[2];  
            k[3] 0   -k[1]; 
           -k[2] k[1] 0 ]
end 

cross_prod_matrix (generic function with 1 method)

In [2]:
function rot_matrix_3d_transform( k, theta )
    mat0 = [ 1 0          0; 
             0 cos(theta) -sin(theta); 
             0 sin(theta) cos( theta )]
    
    if isapprox( k[1], 0.0 ) && isapprox( k[2], 0.0 )
        u = [ 0.0, 0.0, 1.0 ]
    else 
        u = [ k[2], -k[1], 0 ]
        u /= norm( u )
    end 
    
    v = cross( k, u )
    trans = hcat( k, u, v )
    return trans*mat0*transpose( trans )
end 

rot_matrix_3d_transform (generic function with 1 method)

In [3]:
function rot_matrix_3d( k, theta )
    
    K = cross_prod_matrix( k )
    return I + sin(theta)K + (1-cos(theta))K^2
end 

rot_matrix_3d (generic function with 1 method)

In [4]:
function rot_matrix_infinitesimal_3d( k, theta )
    K = cross_prod_matrix( k )
    return I+theta*K
end

rot_matrix_infinitesimal_3d (generic function with 1 method)

In [5]:
rot_x_theta( theta ) = rot_matrix_3d( [1.0, 0.0, 0.0], theta )
rot_y_theta( theta ) = rot_matrix_3d( [0.0, 1.0, 0.0], theta )
rot_z_theta( theta ) = rot_matrix_3d( [0.0, 0.0, 1.0], theta )    

rot_z_theta (generic function with 1 method)

In [6]:
function rand_rot_3d()

    k = [ rand()- .5 for x in 1:3 ]
    k = k/norm(k)
    theta = 2*pi*rand()
    return rot_matrix_3d( k, theta )
end

rand_rot_3d (generic function with 1 method)

In [7]:
function params_rotation_3d( R )
   
    theta = acos( (R[1,1] + R[2,2] + R[3,3] -1 )/2 )
    k1 = (R[3,2]-R[2,3])/(2*sin(theta))
    k2 = (R[1,3]-R[3,1])/(2*sin(theta))
    k3 = (R[2,1]-R[1,2])/(2*sin(theta))
    
    return [k1, k2, k3], theta
end

params_rotation_3d (generic function with 1 method)

In [8]:
function zxz_angles( R ) 
    
    theta = acos( R[3,3] )
    
    phi = pi/2#asin( R[3,1]/sin(theta) ); 
    if !isapprox( R[3,2], sin(theta)*cos(phi))
            phi = pi-phi
    end
    
    psi = -pi/2#asin( R[1,3]/sin(theta))
    if !isapprox( R[2,3], -cos(psi)*sin(theta)) 
        psi = pi-psi
    end 
    
    return phi, theta, psi
end

zxz_angles (generic function with 1 method)

In [9]:
function zyx_angles( R )
    theta = asin( -R[3,1] )
    phi = asin( R[3,2]/cos(theta))
    
    if !isapprox( cos(phi)*cos(theta), R[3,3] )
        phi = pi-phi
    end 
    
    psi = asin( R[2,1]/cos(theta))
    
    if !isapprox( cos(theta)*cos(psi), R[1,1])
        psi = pi-psi
    end
    
    return phi, theta, psi
end

zyx_angles (generic function with 1 method)

In [10]:
R = rand_rot_3d()

3×3 Matrix{Float64}:
 -0.389256   -0.219506  0.894593
  0.919513   -0.035079  0.391491
 -0.0545531   0.97498   0.215493

In [11]:
phi, theta, psi = zyx_angles( R )
rot_z_theta( psi )*rot_y_theta( theta )*rot_x_theta( phi )

3×3 Matrix{Float64}:
 -0.389256   -0.219506  0.894593
  0.919513   -0.035079  0.391491
 -0.0545531   0.97498   0.215493

In [12]:
phi, theta, psi = zxz_angles( R )
rot_z_theta( psi )*rot_x_theta( theta )*rot_z_theta( phi )

3×3 Matrix{Float64}:
  0.215493     2.45969e-16  -0.976505
 -1.58871e-16  1.0           2.16828e-16
  0.976505     1.08414e-16   0.215493

In [13]:
r = rot_y_theta( 2pi*rand() )

3×3 Matrix{Float64}:
 -0.936347  0.0   0.351077
  0.0       1.0   0.0
 -0.351077  0.0  -0.936347

In [14]:
phi, theta, psi = zxz_angles( r )
rot_z_theta( psi )*rot_x_theta( theta )*rot_z_theta( phi )

3×3 Matrix{Float64}:
 -0.936347     1.18089e-16  -0.351077
  9.68884e-17  1.0           7.79548e-17
  0.351077     3.89774e-17  -0.936347

In [15]:
isapprox(1.0000000000000002, 1 )

true

In [16]:
phi, theta, psi

(1.5707963267948966, 2.7828715592524, 4.71238898038469)

In [17]:
pi/2

1.5707963267948966

In [32]:
k =[ 2*(rand()-.5) for i in 1:3 ]
theta = .01
r1 = rot_matrix_3d( k, theta )

3×3 Matrix{Float64}:
  0.999999     -0.000890185   0.000867494
  0.000896466   0.999973     -0.00726713
 -0.000861002   0.0072679     0.999973

In [33]:
r2 = rot_matrix_infinitesimal_3d( k, theta )

3×3 Matrix{Float64}:
  1.0          -0.00089334   0.000864262
  0.00089334    1.0         -0.00726764
 -0.000864262   0.00726764   1.0

In [20]:
det(r1)

0.9999999999999399

In [21]:
det(r2)

1.0000003999209655

In [34]:
r1 = rot_matrix_infinitesimal_3d( [1,0,0], 0.001 )
r2 = rot_matrix_infinitesimal_3d( [0,1,0], 0.001 )
r3 = rot_matrix_infinitesimal_3d( [0,0,1], 0.001 )


3×3 Matrix{Float64}:
 1.0    -0.001  0.0
 0.001   1.0    0.0
 0.0     0.0    1.0

In [39]:
k = [-1,2,3]
k /= norm(k)
k

3-element Vector{Float64}:
 -0.2672612419124244
  0.5345224838248488
  0.8017837257372732

In [41]:
r1^-1*r2^2*r3^3

3×3 Matrix{Float64}:
  0.999996    -0.003        0.002
  0.002998     0.999996     0.000999998
 -0.00200299  -0.000993996  0.999998

In [42]:
rot_matrix_infinitesimal_3d( [-1,2,3], 0.001 )

3×3 Matrix{Float64}:
  1.0    -0.003  0.002
  0.003   1.0    0.001
 -0.002  -0.001  1.0

In [43]:
rot_matrix_3d( [-1,2,3], 0.001 )

3×3 Matrix{Float64}:
  0.999994   -0.003001  0.0019985
  0.002999    0.999995  0.001003
 -0.0020015  -0.000997  0.999998